In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [ ]:
# 데이터에서 좌표정보만 불러오기
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/1석 2조/geofinal.xlsx', usecols=['좌표정보(X)', '좌표정보(Y)'])

In [ ]:
# 불러온 데이터 확인
data

,좌표정보(X),좌표정보(Y)
0,201643.277883,452339.843126
1,199003.460155,452574.992954
2,198988.983646,452223.097221
3,200280.218940,452611.575937
4,198498.918633,452473.403750
...,...,...
67005,222948.740000,308096.375000
67006,NaN,NaN
67007,222926.016113,308227.488983
67008,222939.825046,308049.683343


# 좌표 변환 라이브러리 install

In [ ]:
!pip install pyproj

     |████████████████████████████████| 6.3 MB 23.0 MB/s 


In [ ]:
# 좌표계 변환하기 위하여 라이브러리 불러오기
import numpy as np
import pyproj
import folium

In [ ]:
data['좌표정보(X)'] = pd.to_numeric(data['좌표정보(X)'], errors="coerce")
data['좌표정보(Y)'] = pd.to_numeric(data['좌표정보(Y)'], errors="coerce")

data = data.dropna()
data.index=range(len(data))
data

,좌표정보(X),좌표정보(Y)
0,201643.277883,452339.843126
1,199003.460155,452574.992954
2,198988.983646,452223.097221
3,200280.218940,452611.575937
4,198498.918633,452473.403750
...,...,...
62689,222795.389833,308221.529382
62690,222948.740000,308096.375000
62691,222926.016113,308227.488983
62692,222939.825046,308049.683343


## 좌표변환

In [ ]:
# 좌표계로 변환해주는 함수
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [ ]:
# DataFrame -> NumPy Array 변환
coord = np.array(data)
coord

array([[201643.27788272, 452339.84312565],
       [199003.46015545, 452574.99295403],
       [198988.98364622, 452223.09722107],
       ...,
       [222926.016113  , 308227.488983  ],
       [222939.825046  , 308049.683343  ],
       [222944.69968   , 308210.624473  ]])

In [ ]:
# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # This is added ba

array([[127.01649963,  37.57334837],
       [126.98661668,  37.57546778],
       [126.98645332,  37.57229723],
       ...,
       [127.25306594,  36.27450138],
       [127.25321443,  36.27289871],
       [127.25327339,  36.27434896]])

In [ ]:
# 경도 위도 설정
data['경도'] = result[:, 0]
data['위도'] = result[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data.tail()

,좌표정보(X),좌표정보(Y),경도,위도
62689,222795.389833,308221.529382,127.251612,36.274451
62690,222948.740000,308096.375000,127.253315,36.273319
62691,222926.016113,308227.488983,127.253066,36.274501
62692,222939.825046,308049.683343,127.253214,36.272899
62693,222944.699680,308210.624473,127.253273,36.274349


In [ ]:
df = pd.read_excel('/content/drive/Shareddrives/쥰나쎄/권나경/geofinal.xlsx')

In [ ]:
df = df[['시도명','시군구명','개방서비스명', '개방자치단체코드', '인허가일자', '영업상태구분코드', '폐업일자', '휴업시작일자', '사업장명', '좌표정보(X)','소재지면적']]
data = data[['좌표정보(X)', '경도', '위도']]

In [ ]:
# 좌표정보(X)의 결측값을 포함한 열을 제거해주기
df1 = df.dropna(subset=['좌표정보(X)'])

In [ ]:
# 인덱스 재정렬 해주기
df2 = df1.reset_index()

In [ ]:
# 필요한 칼럼만 추출해주기
df3 = df2[['시도명','시군구명','개방서비스명', '개방자치단체코드', '인허가일자', '영업상태구분코드', '폐업일자', '휴업시작일자', '사업장명', '좌표정보(X)','소재지면적']]

In [ ]:
df3

,시도명,시군구명,개방서비스명,개방자치단체코드,인허가일자,영업상태구분코드,폐업일자,휴업시작일자,사업장명,좌표정보(X),소재지면적
0,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,201643.277883,NaN
1,서울특별시,종로구,관광숙박업,3000000,20180618,3,20201008.0,NaN,JONGRO ALICE,199003.460155,NaN
2,서울특별시,종로구,관광숙박업,3000000,20191029,1,NaN,NaN,목시 서울 인사동 호텔,198988.983646,NaN
3,서울특별시,종로구,관광숙박업,3000000,20191018,1,NaN,NaN,글루호텔,200280.218940,NaN
4,서울특별시,종로구,관광숙박업,3000000,20191002,1,NaN,NaN,나인트리 프리미어 호텔 인사동,198498.918633,NaN
...,...,...,...,...,...,...,...,...,...,...,...
62689,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,222795.389833,"2,040.65"
62690,충청남도,계룡시,숙박업,5580000,20030227,3,20170530.0,NaN,스파소모텔,222948.740000,"1,567.97"
62691,충청남도,계룡시,숙박업,5580000,20030609,3,20140325.0,NaN,폴라리스모텔,222926.016113,NaN
62692,충청남도,계룡시,숙박업,5580000,20030528,3,20170222.0,NaN,쏘울모텔,222939.825046,"1,999.00"


In [ ]:
data

,좌표정보(X),경도,위도
0,201643.277883,127.016500,37.573348
1,199003.460155,126.986617,37.575468
2,198988.983646,126.986453,37.572297
3,200280.218940,127.001070,37.575798
4,198498.918633,126.980905,37.574552
...,...,...,...
62689,222795.389833,127.251612,36.274451
62690,222948.740000,127.253315,36.273319
62691,222926.016113,127.253066,36.274501
62692,222939.825046,127.253214,36.272899


In [ ]:
# 데이터 병합하기
DATA = pd.merge(df3,
                data,
                how = 'right',
                on = '좌표정보(X)')
DATA

,시도명,시군구명,개방서비스명,개방자치단체코드,인허가일자,영업상태구분코드,폐업일자,휴업시작일자,사업장명,좌표정보(X),소재지면적,경도,위도
0,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,201643.277883,NaN,127.016500,37.573348
1,서울특별시,종로구,숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,201643.277883,"2,908.49",127.016500,37.573348
2,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,201643.277883,NaN,127.016500,37.573348
3,서울특별시,종로구,숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,201643.277883,"2,908.49",127.016500,37.573348
4,서울특별시,종로구,관광숙박업,3000000,20180618,3,20201008.0,NaN,JONGRO ALICE,199003.460155,NaN,126.986617,37.575468
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91581,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,222795.389833,"2,040.65",127.251612,36.274451
91582,충청남도,계룡시,숙박업,5580000,20200515,1,NaN,NaN,화이트모텔,222795.389833,"1,746.38",127.251612,36.274451
91583,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,222795.389833,"2,040.65",127.251612,36.274451
91584,충청남도,계룡시,숙박업,5580000,20030227,3,20210714.0,NaN,호텔리어,221864.221211,"1,905.13",127.241293,36.288753


In [ ]:
# 필요한 컬럼만 추출하기
inhega = DATA[['시도명','시군구명','개방서비스명', '개방자치단체코드', '인허가일자', '영업상태구분코드', '폐업일자', '휴업시작일자', '사업장명', '경도','위도','소재지면적']]

In [ ]:
inhega

,시도명,시군구명,개방서비스명,개방자치단체코드,인허가일자,영업상태구분코드,폐업일자,휴업시작일자,사업장명,경도,위도,소재지면적
0,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,NaN
1,서울특별시,종로구,숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,"2,908.49"
2,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,NaN
3,서울특별시,종로구,숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,"2,908.49"
4,서울특별시,종로구,관광숙박업,3000000,20180618,3,20201008.0,NaN,JONGRO ALICE,126.986617,37.575468,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
91581,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,127.251612,36.274451,"2,040.65"
91582,충청남도,계룡시,숙박업,5580000,20200515,1,NaN,NaN,화이트모텔,127.251612,36.274451,"1,746.38"
91583,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,127.251612,36.274451,"2,040.65"
91584,충청남도,계룡시,숙박업,5580000,20030227,3,20210714.0,NaN,호텔리어,127.241293,36.288753,"1,905.13"


In [ ]:
# https://mizykk.tistory.com/93
# 중복된 열을 제거 해주기
final = inhega.drop_duplicates(ignore_index = True)

In [ ]:
final

,시도명,시군구명,개방서비스명,개방자치단체코드,인허가일자,영업상태구분코드,폐업일자,휴업시작일자,사업장명,경도,위도,소재지면적
0,서울특별시,종로구,관광숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,NaN
1,서울특별시,종로구,숙박업,3000000,20201023,1,NaN,NaN,호텔 DM,127.016500,37.573348,"2,908.49"
2,서울특별시,종로구,관광숙박업,3000000,20180618,3,20201008.0,NaN,JONGRO ALICE,126.986617,37.575468,NaN
3,서울특별시,종로구,숙박업,3000000,20180618,3,20201008.0,NaN,JONGRO ALICE,126.986617,37.575468,278.43
4,서울특별시,종로구,관광숙박업,3000000,20191029,1,NaN,NaN,목시 서울 인사동 호텔,126.986453,37.572297,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
62664,충청남도,계룡시,숙박업,5580000,20030528,3,20170222.0,NaN,쏘울모텔,127.253214,36.272899,"1,999.00"
62665,충청남도,계룡시,숙박업,5580000,20200515,1,NaN,NaN,화이트모텔,127.251612,36.274451,"1,746.38"
62666,충청남도,계룡시,숙박업,5580000,20030814,3,20200314.0,NaN,화이트모텔,127.251612,36.274451,"2,040.65"
62667,충청남도,계룡시,숙박업,5580000,20030227,3,20210714.0,NaN,호텔리어,127.241293,36.288753,"1,905.13"


In [ ]:
final = final[final.영업상태구분코드 != 4]

In [ ]:
final.to_excel('jjinjjinfinal.xlsx')